# 🧠 Episodic Memory Demo

> **Give your agents memory that persists and learns.**

## Learning Objectives

By the end of this notebook, you will:
1. Understand episodic memory vs. working memory
2. Store and retrieve agent episodes
3. Use semantic search for relevant memories
4. Implement memory compression (sleep cycle)
5. Track failures with negative memory

---

## Why Episodic Memory?

**Problem:** Agents without memory repeat mistakes and can't learn from experience.

**Solution:** An immutable, append-only ledger of agent experiences.

```
Traditional DB:     Episodic Memory:
┌─────────────┐     ┌─────────────┐
│ UPDATE ✓    │     │ APPEND ONLY │
│ DELETE ✓    │     │ NO UPDATE   │
│ MODIFY ✓    │     │ NO DELETE   │
└─────────────┘     └─────────────┘
     ↓                    ↓
  Audit gaps          Full history
  State bugs          Time-travel OK
```

---

## Step 1: Install Dependencies

In [ ]:
!pip install agent-os emk --quiet

## Step 2: Create Your First Episode

In [ ]:
from emk import Episode, FileAdapter
from datetime import datetime

# Initialize storage (uses a JSONL file)
store = FileAdapter("demo_memory.jsonl")

# Create an episode
episode = Episode(
    goal="Query customer data for Q4 analysis",
    action="SELECT * FROM customers WHERE quarter='Q4' AND year=2024",
    result="Retrieved 1,523 customer records in 0.3s",
    reflection="Query was efficient. Index on quarter+year helped."
)

# Store it (immutable - can never be changed)
store.store(episode)

print("✅ Episode stored!")
print(f"   ID: {episode.id}")
print(f"   Timestamp: {episode.timestamp}")
print(f"   Goal: {episode.goal}")

## Step 3: Store More Episodes

Let's add several episodes to build up memory:

In [ ]:
# Simulate a series of agent experiences
episodes = [
    Episode(
        goal="Analyze sales trends",
        action="Aggregated sales by region and product category",
        result="Found 15% increase in West region, electronics leading",
        reflection="Regional breakdown was valuable. Should do this monthly."
    ),
    Episode(
        goal="Generate executive summary",
        action="Summarized Q4 performance for board presentation",
        result="3-page summary with charts delivered",
        reflection="Executive prefers bullet points over prose."
    ),
    Episode(
        goal="Debug slow query",
        action="Added index on orders.customer_id column",
        result="Query time reduced from 12s to 0.2s",
        reflection="Always check for missing indexes first."
    ),
    Episode(
        goal="Send weekly report email",
        action="Attempted to send via SMTP",
        result="FAILED: Connection timeout after 30s",
        reflection="SMTP server unreliable. Should use API instead."
    ),
]

for ep in episodes:
    store.store(ep)
    print(f"✅ Stored: {ep.goal[:40]}...")

print(f"\n📊 Total episodes in memory: {len(store)}")

## Step 4: Retrieve Memories

### Simple Retrieval (All Episodes)

In [ ]:
# Get all episodes
all_episodes = store.list_all()

print("📜 All Episodes:")
print("-" * 60)
for ep in all_episodes:
    status = "❌" if "FAILED" in ep.result else "✅"
    print(f"{status} {ep.timestamp.strftime('%Y-%m-%d %H:%M')} | {ep.goal[:40]}")

### Semantic Search (Find Similar Memories)

Retrieve episodes relevant to a current task:

In [ ]:
# Search for relevant memories
query = "How to optimize slow database queries?"
similar = store.retrieve(query=query, k=3)

print(f"🔍 Query: {query}")
print("\n📚 Relevant memories:")
print("-" * 60)
for ep in similar:
    print(f"\n🎯 Goal: {ep.goal}")
    print(f"   Action: {ep.action}")
    print(f"   Learning: {ep.reflection}")

## Step 5: Negative Memory (Tracking Failures)

Explicitly track failures to avoid repeating mistakes:

In [ ]:
# Create an episode and mark it as a failure
failed_episode = Episode(
    goal="Call external API for weather data",
    action="GET https://api.weather.old/v1/forecast",
    result="FAILED: 404 Not Found - API deprecated",
    reflection="This API no longer exists. Use new endpoint."
)

# Mark as failure with reason
failed_episode = failed_episode.mark_as_failure(
    reason="API endpoint deprecated since Jan 2024"
)

store.store(failed_episode)
print("❌ Failure recorded!")
print(f"   Reason: {failed_episode.failure_reason}")

In [ ]:
# Retrieve both successes and failures
patterns = store.retrieve_with_anti_patterns()

print("📊 Memory Analysis:")
print(f"   ✅ Successes: {len(patterns['successes'])}")
print(f"   ❌ Failures: {len(patterns['failures'])}")

if patterns['failures']:
    print("\n⚠️  Things to avoid:")
    for fail in patterns['failures']:
        print(f"   • {fail.goal}: {fail.failure_reason}")

## Step 6: Memory Compression (Sleep Cycle)

Over time, memory grows. Compress old episodes into semantic rules:

In [ ]:
from emk import MemoryCompressor

# Initialize compressor
compressor = MemoryCompressor(
    store=store,
    age_threshold_days=0  # Compress all for demo (normally 30+ days)
)

# Run compression
result = compressor.compress_old_episodes()

print("🌙 Sleep Cycle Complete!")
print(f"   Episodes processed: {result['episodes_processed']}")
print(f"   Rules generated: {result['rules_generated']}")
print(f"   Storage saved: {result['storage_saved_percent']}%")

In [ ]:
# View generated rules
print("📖 Semantic Rules (distilled knowledge):")
print("-" * 60)
for rule in result['rules']:
    print(f"\n• {rule['pattern']}")
    print(f"  → {rule['insight']}")

## Step 7: Integrate with Agent OS

Use episodic memory in a governed agent:

In [ ]:
from agent_os import KernelSpace

kernel = KernelSpace(policy="strict")
memory = FileAdapter("agent_memories.jsonl")

@kernel.register
async def learning_agent(task: str):
    """
    An agent that learns from its past experiences.
    """
    # 1. Check memory for similar past tasks
    relevant = memory.retrieve(query=task, k=3)
    
    # 2. Build context from past experiences
    context = "\n".join([
        f"Past: {ep.goal} → {ep.reflection}" 
        for ep in relevant
    ])
    
    # 3. Execute task (your LLM logic here)
    result = f"Processed '{task}' with context from {len(relevant)} memories"
    
    # 4. Record this episode
    episode = Episode(
        goal=task,
        action="task_execution",
        result=result,
        reflection="Successfully used memory context"
    )
    memory.store(episode)
    
    return result

# Execute
result = await kernel.execute(learning_agent, "Optimize the sales dashboard")
print(f"\n📤 Result: {result}")

## Cleanup

In [ ]:
import os

# Remove demo files
for f in ["demo_memory.jsonl", "agent_memories.jsonl"]:
    if os.path.exists(f):
        os.remove(f)
        print(f"🗑️  Removed {f}")

---

## Summary

| Feature | What It Does |
|---------|-------------|
| `Episode` | Single unit of agent experience |
| `FileAdapter` | Stores episodes in JSONL |
| `store.retrieve()` | Semantic search for relevant memories |
| `mark_as_failure()` | Track failures for anti-patterns |
| `MemoryCompressor` | Compress old memories into rules |

### Quick Reference

```python
from emk import Episode, FileAdapter, MemoryCompressor

# Store
store = FileAdapter("memory.jsonl")
store.store(Episode(goal="...", action="...", result="...", reflection="..."))

# Retrieve
similar = store.retrieve(query="...", k=5)

# Failures
ep = ep.mark_as_failure(reason="...")

# Compress
compressor = MemoryCompressor(store, age_threshold_days=30)
compressor.compress_old_episodes()
```

---

## Next Steps

- [03-time-travel-debugging](03-time-travel-debugging.ipynb) - Replay agent decisions
- [04-cross-model-verification](04-cross-model-verification.ipynb) - Detect hallucinations
- [EMK Documentation](https://github.com/imran-siddique/emk)